In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import logging

import pandas as pd
import tensorflow.keras as keras

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model

# Log setting
logging.basicConfig(format="%(asctime)s %(levelname)s %(message)s", datefmt="%H:%M:%S", level=logging.INFO)

# Change display.max_rows to show all features.
pd.set_option("display.max_rows", 85)

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers

In [ ]:
training_df = pd.read_csv('/content/drive/MyDrive/train_MachineLearningCVE.csv', skipinitialspace=True)
testing_df = pd.read_csv('/content/drive/MyDrive/test_MachineLearningCVE.csv', skipinitialspace=True)

In [ ]:
print("Training set has {} rows.".format(len(training_df)))
print("Testing set has {} rows.".format(len(testing_df)))

Training set has 2264594 rows.
Testing set has 566149 rows.


In [ ]:

# Our new labels
classes=["Normal","Benign","FTP-Patator","SSH-Patator","DoS slowloris","DoS Slowhttptest","DoS Hulk","DoS GoldenEye","Heartbleed","Web Attack–Brute force","Web Attack–XSS","Web Attack–SQL Injection","Infiltration","Bot","DDoS","Port scan"]

#Helper function to label samples to 5 classes
def label_attack (row):
    if row["Label"] == 0:
        return classes[1]
    if row["Label"] == 1:
        return classes[2]
    if row["Label"] == 2:
        return classes[3]
    if row["Label"] == 3:
        return classes[4]
    if row["Label"] == 4:
        return classes[5]
    if row["Label"] == 4:
        return classes[5]
    if row["Label"] == 5:
        return classes[6]
    if row["Label"] == 6:
        return classes[7]
    if row["Label"] == 7:
        return classes[8]
    if row["Label"] == 8:
        return classes[9]
    if row["Label"] == 9:
        return classes[10]
    if row["Label"] == 10:
        return classes[11]
    if row["Label"] == 11:
        return classes[12]
    if row["Label"] == 12:
        return classes[13]
    if row["Label"] == 13:
        return classes[14]
    if row["Label"] == 12:
        return classes[13]
    if row["Label"] == 13:
        return classes[14]
    if row["Label"] == 14:
        return classes[15]
    return classes[0]


In [ ]:
test_samples_length = len(testing_df)
df=pd.concat([training_df,testing_df])
df["Class"]=df.apply(label_attack,axis=1)

In [ ]:
df=df.drop("Label",axis=1)

In [ ]:
training_df= df.iloc[:-test_samples_length, :]
testing_df= df.iloc[-test_samples_length:,:]

In [ ]:
training_outcomes=training_df["Class"].unique()
testing_outcomes=testing_df["Class"].unique()
print("The training set has {} possible outcomes \n".format(len(training_outcomes)) )
print(", ".join(training_outcomes)+".")
print("\nThe testing set has {} possible outcomes \n".format(len(testing_outcomes)))
print(", ".join(testing_outcomes)+".")

The training set has 15 possible outcomes 

Benign, Web Attack–SQL Injection, DoS Slowhttptest, Heartbleed, DoS slowloris, DoS Hulk, DoS GoldenEye, Infiltration, FTP-Patator, Bot, Port scan, Web Attack–XSS, Web Attack–Brute force, DDoS, SSH-Patator.

The testing set has 15 possible outcomes 

Benign, DoS Slowhttptest, Web Attack–SQL Injection, DoS slowloris, Heartbleed, DoS GoldenEye, DoS Hulk, Infiltration, Bot, FTP-Patator, Port scan, Web Attack–XSS, Web Attack–Brute force, DDoS, SSH-Patator.


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers import Input,Dropout,Dense
from keras.models import Model
from keras import regularizers
from keras.utils.data_utils import get_file
%matplotlib inline

In [ ]:
# Helper function for scaling continous values
def minmax_scale_values(training_df,testing_df, col_name):
    scaler = MinMaxScaler()
    scaler = scaler.fit(training_df[col_name].values.reshape(-1, 1))
    train_values_standardized = scaler.transform(training_df[col_name].values.reshape(-1, 1))
    training_df[col_name] = train_values_standardized
    test_values_standardized = scaler.transform(testing_df[col_name].values.reshape(-1, 1))
    testing_df[col_name] = test_values_standardized
    
    
#Helper function for one hot encoding
def encode_text(training_df,testing_df, name):
    training_set_dummies = pd.get_dummies(training_df[name])
    testing_set_dummies = pd.get_dummies(testing_df[name])
    for x in training_set_dummies.columns:
        dummy_name = "{}_{}".format(name, x)
        training_df[dummy_name] = training_set_dummies[x]
        if x in testing_set_dummies.columns :
            testing_df[dummy_name]=testing_set_dummies[x]
        else :
            testing_df[dummy_name]=np.zeros(len(testing_df))
    training_df.drop(name, axis=1, inplace=True)
    testing_df.drop(name, axis=1, inplace=True)
    
    
sympolic_columns=["protocol_type","service","flag"]
label_column="Class"
for column in df.columns :
    if column in sympolic_columns:
        encode_text(training_df,testing_df,column)
    elif not column == label_column:
        minmax_scale_values(training_df,testing_df, column)

In [ ]:
x,y=training_df,training_df.pop("Class").values
x=x.values
x_test,y_test=testing_df,testing_df.pop("Class").values
x_test=x_test.values
y0=np.ones(len(y),np.int8)
y0[np.where(y==classes[0])]=0
y0_test=np.ones(len(y_test),np.int8)
y0_test[np.where(y_test==classes[0])]=0

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras import regularizers

import tqdm
import tensorflow as tf

In [ ]:
from keras.losses import binary_crossentropy, mean_squared_error
from keras import backend as K

def L(X, X_, t):
  if t == 'binary':
    return binary_crossentropy(X, X_)
  return mean_squared_error(X, X_)

def R(X):
  return K.dot(X, K.transpose(X))

def tau(X, t):
  return tf.where(X < t, X, tf.zeros(tf.shape(X)))

def rae_loss(alpha, t, L_type='binary'):
  def rae(y_true, y_pred):
    return (1 - alpha)*L(y_true, y_pred, L_type) + alpha*L(tau(R(y_true), t), tau(R(y_pred), t), L_type)
  return rae

In [ ]:
def plot_digits(n, X, decoded_X):
    plt.figure(figsize=(2*n, 4))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(X[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_X[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
inputs = Input(shape=(78,))

encoded = Dense(units=128, activation='relu')(inputs)

encoded = Dense(units=64, activation='relu')(encoded)

encoded = Dense(units=2, activation='relu')(encoded)

decoded = Dense(units=64, activation='relu')(encoded)

decoded = Dense(units=128, activation='relu')(decoded)

decoded = Dense(units=78, activation='sigmoid')(decoded)

In [ ]:
autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

In [ ]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 78)]              0         
                                                                 
 dense (Dense)               (None, 128)               10112     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
 dense_3 (Dense)             (None, 64)                192       
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 78)                10062 

In [ ]:
alphas = np.linspace(0, 1, 4)
t = 1
losses = []

for alpha in alphas:
    loss = rae_loss(alpha, t, 'binary')
    autoencoder.compile(optimizer='adam', loss=loss)
    
    print('Alpha {}'.format(alpha))
    
    h = autoencoder.fit(x[np.where(y0==1)],x[np.where(y0==1)],
                    epochs=2,
                    batch_size=100,
                    shuffle=True, 
                    validation_split=0.1)
    
    losses.append(h.history['val_loss'][-1])
    print(h.history['val_loss'][-1])
    

Alpha 0.0
Epoch 1/2
20382/20382 [==============================] - 87s 4ms/step - loss: 0.0487 - val_loss: 0.0628
Epoch 2/2
20382/20382 [==============================] - 84s 4ms/step - loss: 0.0449 - val_loss: 0.0602
0.060158323496580124
Alpha 0.3333333333333333
Epoch 1/2
20382/20382 [==============================] - 86s 4ms/step - loss: 0.1162 - val_loss: 0.1381
Epoch 2/2
20382/20382 [==============================] - 86s 4ms/step - loss: 0.1132 - val_loss: 0.1396
0.1395607888698578
Alpha 0.6666666666666666
Epoch 1/2
20382/20382 [==============================] - 90s 4ms/step - loss: 0.1491 - val_loss: 0.1668
Epoch 2/2
20382/20382 [==============================] - 87s 4ms/step - loss: 0.1487 - val_loss: 0.1649
0.1649409979581833
Alpha 1.0
Epoch 1/2
20382/20382 [==============================] - 86s 4ms/step - loss: 0.1773 - val_loss: 0.1894
Epoch 2/2
20382/20382 [==============================] - 94s 5ms/step - loss: 0.1769 - val_loss: 0.1922
0.1921989917755127
